# Preprocess

In [1]:
import pandas as pd
import numpy as np

### Read the csv file.

In [2]:
orders_df = pd.read_csv("data/archive/olist_orders_dataset.csv")
customers_df = pd.read_csv("data/archive/olist_customers_dataset.csv")
geolocation_df = pd.read_csv("data/archive/olist_geolocation_dataset.csv")
order_items_df = pd.read_csv("data/archive/olist_order_items_dataset.csv")
products_df = pd.read_csv("data/archive/olist_products_dataset.csv")
sellers_df = pd.read_csv("data/archive/olist_sellers_dataset.csv")

### Get the label column by comparing df['order_estimated_delivery_date'] and  df['order_delivered_customer_date']

In [3]:
orders_df.order_purchase_timestamp = pd.to_datetime(pd.to_datetime(orders_df.order_purchase_timestamp, format='%Y-%m-%d %H:%M:%S').dt.date)
orders_df.order_approved_at = pd.to_datetime(pd.to_datetime(orders_df.order_approved_at, format='%Y-%m-%d %H:%M:%S').dt.date)
orders_df.order_delivered_carrier_date = pd.to_datetime(pd.to_datetime(orders_df.order_delivered_carrier_date, format='%Y-%m-%d %H:%M:%S').dt.date)
orders_df.order_delivered_customer_date = pd.to_datetime(pd.to_datetime(orders_df.order_delivered_customer_date, format='%Y-%m-%d %H:%M:%S').dt.date)
orders_df.order_estimated_delivery_date = pd.to_datetime(pd.to_datetime(orders_df.order_estimated_delivery_date, format='%Y-%m-%d %H:%M:%S').dt.date)

In [4]:
orders_df = orders_df[orders_df['order_status'] == 'delivered']
orders_df.drop('order_status', axis=1, inplace=True)
orders_df['anomaly'] = orders_df['order_estimated_delivery_date'] < orders_df['order_delivered_customer_date']

### Enforce a rule: order_purchase_timestamp <= order_approved_at <= order_delivered_carrier_date <= order_delivered_customer_date

In [5]:
orders_df = orders_df[orders_df['order_purchase_timestamp'] <= orders_df['order_approved_at']]
orders_df = orders_df[orders_df['order_approved_at'] <= orders_df['order_delivered_carrier_date']]
orders_df = orders_df[orders_df['order_delivered_carrier_date'] <= orders_df['order_delivered_customer_date']]

Time to approve an order and to deliver an order to a carrier influences delivery time.

In [6]:
orders_df['time_estimate_delivery'] = (orders_df['order_estimated_delivery_date'] - orders_df['order_delivered_carrier_date']).dt.days

In [7]:
orders_df['year'] = orders_df['order_purchase_timestamp'].dt.year
orders_df['month'] = orders_df['order_purchase_timestamp'].dt.month
orders_df['day'] = orders_df['order_purchase_timestamp'].dt.day

In [8]:
orders_df.drop(['order_approved_at', 'order_delivered_customer_date', 'order_estimated_delivery_date'], axis=1, inplace=True)

In [9]:
orders_df['anomaly'].sum() / orders_df.shape[0]

0.06807573020331868

About 6.8% of the records are anomalies.

### Preprocess the customers dataset.

In [10]:
customers_df.drop(['customer_unique_id', 'customer_city', 'customer_state'], axis=1, inplace=True)

### Preprocess the sellers dataset.

In [11]:
sellers_df.drop(['seller_city', 'seller_state'], axis=1, inplace=True)
sellers_df.head()

,seller_id,seller_zip_code_prefix
0,3442f8959a84dea7ee197c632cb2df15,13023
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195
4,51a04a8a6bdcb23deccc82b0b80742cf,12914


### Preprocess the geolocation dataset.

In [12]:
geolocation_df.drop(['geolocation_city', 'geolocation_state'], axis=1, inplace=True)
geolocation_df = geolocation_df.groupby(['geolocation_zip_code_prefix']).mean()

In [13]:
geolocation_df.head()

,geolocation_lat,geolocation_lng
geolocation_zip_code_prefix,,
1001,-23.550190,-46.634024
1002,-23.548146,-46.634979
1003,-23.548994,-46.635731
1004,-23.549799,-46.634757
1005,-23.549456,-46.636733


### Merge customers and sellers data with geolocation information.

In [14]:
customers_geolocation_df = pd.merge(customers_df, geolocation_df, how="left", left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')
customers_geolocation_df.dropna(inplace=True)
customers_geolocation_df.rename(columns={"geolocation_lat": "customer_geolocation_lat", "geolocation_lng": "customer_geolocation_lng"}, inplace=True)

In [15]:
customers_geolocation_df.head()

,customer_id,customer_zip_code_prefix,customer_geolocation_lat,customer_geolocation_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,-20.498489,-47.396929
1,18955e83d337fd6b2def6b18a428ac77,9790,-23.727992,-46.542848
2,4e7b3e00288586ebd08712fdd0374a03,1151,-23.531642,-46.656289
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,-23.499702,-46.185233
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,-22.975100,-47.142925


In [16]:
sellers_geolocation_df = pd.merge(sellers_df, geolocation_df, how="left", left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix')
sellers_geolocation_df.dropna(inplace=True)
sellers_geolocation_df.rename(columns={"geolocation_lat": "seller_geolocation_lat", "geolocation_lng": "seller_geolocation_lng"}, inplace=True)

In [17]:
sellers_geolocation_df.head()

,seller_id,seller_zip_code_prefix,seller_geolocation_lat,seller_geolocation_lng
0,3442f8959a84dea7ee197c632cb2df15,13023,-22.893848,-47.061337
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,-22.383437,-46.947927
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,-22.909572,-43.177703
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,-23.657242,-46.612831
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,-22.964803,-46.534419


### Preprocess products data

In [18]:
products_df.drop(['product_name_lenght', 'product_description_lenght', 'product_photos_qty'], axis=1, inplace=True)
products_df.product_category_name = products_df.product_category_name.astype(str)
products_df.dropna(inplace=True)

In [19]:
products_df.head()

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,625.0,20.0,17.0,13.0


### Merge order items data with sellers and product information

In [20]:
order_items_df.shipping_limit_date = pd.to_datetime(pd.to_datetime(order_items_df.shipping_limit_date, format='%Y-%m-%d %H:%M:%S').dt.date)
order_items_sellers_geolocation_df = pd.merge(order_items_df, sellers_geolocation_df, how='left', on='seller_id')
order_items_sellers_geolocation_df.dropna(inplace=True)
order_items_sellers_geolocation_products_df = pd.merge(order_items_sellers_geolocation_df, products_df, how='left', on='product_id')
order_items_sellers_geolocation_products_df.dropna(inplace=True)

In [21]:
order_items_sellers_geolocation_products_df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_zip_code_prefix,seller_geolocation_lat,seller_geolocation_lng,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19,58.90,13.29,27277.0,-22.496953,-44.127492,cool_stuff,650.0,28.0,9.0,14.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03,239.90,19.93,3471.0,-23.565096,-46.518565,pet_shop,30000.0,50.0,30.0,40.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18,199.00,17.87,37564.0,-22.262584,-46.171124,moveis_decoracao,3050.0,33.0,13.0,33.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15,12.99,12.79,14403.0,-20.553624,-47.387359,perfumaria,200.0,16.0,10.0,15.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13,199.90,18.14,87900.0,-22.929384,-53.135873,ferramentas_jardim,3750.0,35.0,40.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112392,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02,299.99,43.41,88303.0,-26.912574,-48.673980,utilidades_domesticas,10150.0,89.0,15.0,40.0
112393,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20,350.00,36.53,1206.0,-23.535864,-46.642819,informatica_acessorios,8950.0,45.0,26.0,38.0
112394,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30,99.90,16.95,80610.0,-25.469955,-49.289821,esporte_lazer,967.0,21.0,24.0,19.0
112395,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21,55.99,8.72,4733.0,-23.635530,-46.694031,informatica_acessorios,100.0,20.0,20.0,20.0


### Merge orders with order items and customers_geolocation

In [22]:
orders_order_items_sellers_geolocation_products_df = pd.merge(orders_df, order_items_sellers_geolocation_products_df, how='left', on='order_id')
df = pd.merge(orders_order_items_sellers_geolocation_products_df, customers_geolocation_df, how='left', on='customer_id')
df.dropna(inplace=True)

In [23]:
df = df[df.shipping_limit_date >= df.order_delivered_carrier_date]

In [24]:
df.drop(['order_id', 'customer_id', 'order_purchase_timestamp', 'order_delivered_carrier_date', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date'], axis=1, inplace=True)

In [25]:
df.isnull().values.any()

False

In [26]:
df['anomaly'].sum() / df.shape[0]

0.05381306218224482

Percentage of anomalies remains to be around 6.6%.

In [27]:
def haversine(customer_lng, customer_lat, seller_lng, seller_lat):
    lon1, lat1, lon2, lat2 = map(np.radians, [customer_lng, customer_lat, seller_lng, seller_lat])

    lon = seller_lng - customer_lng
    lat = seller_lat - customer_lat

    r = np.sin(lat / 2.0) ** 2 + np.cos(customer_lat) * np.cos(seller_lat) * np.sin(lon / 2.0) ** 2

    distance = 6371 * 2 * np.arcsin(np.sqrt(r))
    return distance

df['distance'] = haversine(df['customer_geolocation_lng'], df['customer_geolocation_lat'], df['seller_geolocation_lng'], df['seller_geolocation_lat'])

In [28]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df[['product_category_name']] = encoder.fit_transform(df[['product_category_name']])

In [29]:
df.to_csv('temp/olist_orders_dataset_df.csv', index=False)

In [30]:
df

,anomaly,time_estimate_delivery,year,month,day,price,freight_value,seller_zip_code_prefix,seller_geolocation_lat,seller_geolocation_lng,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,customer_geolocation_lat,customer_geolocation_lng,distance
0,False,14,2017,10,2,29.99,8.72,9350.0,-23.680729,-46.444238,73.0,500.0,19.0,8.0,13.0,3149.0,-23.576983,-46.587161,662.084641
1,False,18,2018,7,24,118.70,22.76,31570.0,-19.807681,-43.980427,63.0,400.0,19.0,13.0,19.0,47813.0,-12.177924,-44.660711,9344.543745
2,False,27,2018,8,8,159.90,19.22,14840.0,-21.363502,-48.229601,8.0,420.0,24.0,19.0,21.0,75265.0,-16.745150,-48.514783,10713.151535
3,False,23,2017,11,18,45.00,27.20,31842.0,-19.837682,-43.924053,64.0,450.0,30.0,10.0,20.0,59296.0,-5.774190,-35.271143,15422.365811
4,False,12,2018,2,13,19.90,8.72,8752.0,-23.543395,-46.262086,60.0,250.0,51.0,15.0,15.0,9195.0,-23.676370,-46.514627,843.950334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109305,False,23,2018,2,6,174.90,20.10,17602.0,-21.930548,-50.498348,9.0,4950.0,40.0,10.0,40.0,11722.0,-24.001500,-46.449864,8679.619783
109306,False,30,2017,8,27,205.99,65.02,8290.0,-23.553642,-46.452661,29.0,13300.0,32.0,90.0,22.0,45920.0,-17.898358,-39.373630,3984.630835
109307,False,34,2018,1,8,179.99,40.59,37175.0,-20.940578,-45.827237,44.0,6550.0,20.0,20.0,20.0,28685.0,-22.562825,-42.694574,16963.820370
109308,False,34,2018,1,8,179.99,40.59,37175.0,-20.940578,-45.827237,44.0,6550.0,20.0,20.0,20.0,28685.0,-22.562825,-42.694574,16963.820370
